<a href="https://colab.research.google.com/github/kundajelab/revcomp_experiments/blob/master/DropoutTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
!git clone https://github.com/kundajelab/revcomp_experiments.git
%cd revcomp_experiments/
!python setup.py install
%cd ..

Cloning into 'revcomp_experiments'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1390 (delta 6), reused 0 (delta 0), pack-reused 1378
Receiving objects: 100% (1390/1390), 521.46 MiB | 28.92 MiB/s, done.
Resolving deltas: 100% (314/314), done.
Checking out files: 100% (1205/1205), done.
/content/revcomp_experiments
running install
running bdist_egg
running egg_info
creating revcompexp.egg-info
writing revcompexp.egg-info/PKG-INFO
writing dependency_links to revcompexp.egg-info/dependency_links.txt
writing requirements to revcompexp.egg-info/requires.txt
writing top-level names to revcompexp.egg-info/top_level.txt
writing manifest file 'revcompexp.egg-info/SOURCES.txt'
writing manifest file 'revcompexp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg

In [0]:
!pip install concise
!pip install keras-genomics

     |████████████████████████████████| 11.3MB 2.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/6e/9a/e1/2ca53dee91902d0d5a5137be3969e1535623928d08f6c47d58
  Stored in directory: /root/.cache/pip/wheels/c4/27/96/6ba6fe28cbb162c326823553e3e45ac502160d1340566360f8
Successfully built concise gtfparse
  Stored in directory: /root/.cache/pip/wheels/ad/07/53/12c9cab81be5fbb7f824df1cf6d23734f27ce7d52f0675691b
Successfully built keras-genomics


In [0]:
!pip install git+https://github.com/kundajelab/simdna.git@v0.4.3.2#egg=simdna
# import simdna
# !pip install simdna
# import simdna

  Cloning https://github.com/kundajelab/simdna.git (to revision v0.4.3.2) to /tmp/pip-install-4_9iy_1r/simdna
  Running command git clone -q https://github.com/kundajelab/simdna.git /tmp/pip-install-4_9iy_1r/simdna
  Stored in directory: /tmp/pip-ephem-wheel-cache-240jk1un/wheels/ef/4e/d7/9dee95097f6f6732b95c50d89ef356453eee7d455ab545ca77
Successfully built simdna


In [0]:
!pip install simdna

In [0]:
import simdna

In [0]:
import keras 
import keras_genomics
import numpy as np

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.engine import Layer
from keras.models import Sequential 
import keras.layers as k1
from keras.engine.base_layer import InputSpec

from sklearn.model_selection import train_test_split

In [0]:
!motif_density_and_position_sim.py --seed 1234 --motifNames GATA_disc1 TAL1_known1 --max-motifs 4 --min-motifs 1 --mean-motifs 2 --zero-prob 0.5 --seqLength 1000 --posSdevInBp 100 --numSeqs 10000

In [0]:
!head DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata

seqName	sequence	embeddings
synth0	AGCTTAATTTCGGTCGGATTCGACTGCTCGTCTATTATCTAAAGGGAGCGAGGAGTTTTTAGCACACTAATGCAATCGACCGTGTATTGCAGGTCGGTAGTTTGTAAGATCACTTACATTTTTGAACGAACGTAAATACGATTGACTAGATGAATTTTGGAGTTTTGCTTGCCTATCTTTGCTTCACTCCGAATACGATTTTGTTTACACATGGGCCCCTCATTCGCAACAGTTCACTTCAAAGAAAGATCTGAGGATGGGGATTTGCTGCGCAGTTGACAAGAGGTTTAAGTTGCGTGAATCGGAAGTCTAAAAAGTTGTGGGTATAAAAGCTACTCTGTTTATGATAATTGCAGTTTAGCCATGATTCTGTTTACCATCTGTACACTGTAACTTCTACTTAAGTCAGGATTTATTTTACTTGTGATCTCATAGAAGTGTCAATACAGGGTTATGATGGCCTAACAGATGTAATAAACGATATCTCACTAGCGAATTAGTTGAATAGTAGCAAAAACATTTAGTTACGGGTTCATCATAGCAACAGATGGCCGAATTCAAATACGGCCACTTATATAACTTCAGAAGGATTTCCCCGGATTATAGAACACTGGGAGTAGTTCCTAAACAGACCTGGGATCAGATGGTCGACTTCAGCTGTTCAACGGGATTCTAAGTAGAAATCAGGCTACACACGAAATCCTACGTCTGTTTAAAATTTCGAATATTCTCAGCGCGTGTAAGGTTTGATCGGCACTGAGATTGCCTAGCAATTTCAGGTGATTATATTTGCGATACTCGATGCATAAAGGTATACTAATTAGCTAAACAAAATATCTACGGCCCCCCGTGCATAGTTCATGTGCTTGTATTCTGCGTACTGCACTGCGATGTTCTTGGTCATGCATGACGCCCGTTTGGTGTTTGACAATAAGGCCAAAATCTTTTGCTAACCTGAGTATATG

In [0]:
sequences = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").sequences
embeddings = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").embeddings

In [0]:
def check(embeddings): 
    if len(embeddings) > 0:
        all_chars = ""
        for x in embeddings: 
            all_chars += str(x.what)
        if 'TAL' in all_chars and 'GATA' in all_chars: 
            return [1, 1]
        if 'TAL' in all_chars and 'GATA' not in all_chars:
            return [1, 0]
        if 'TAL' not in all_chars and 'GATA' in all_chars:
            return [0, 1]
    return [0,0]

In [0]:
y= np.zeros((0,2))
for x in embeddings: 
    thing = check(x)
    y = np.append(y, [thing], axis=0)

In [0]:
import numpy as np

# def mutate(s, num, target):
#     change_locs = set(sample(range(len(s)), num))
#     changed = (target if i in change_locs else c for i,c in enumerate(s))
#     return ''.join(changed)

def mutate(y, mutation_prob):
    np.random.seed(1234)
    mutated_y = []
    for row in y:
        new_labels_for_row = []
        for label in row:
            if np.random.uniform() < mutation_prob:
                new_labels_for_row.append(1-label)
            else:
                new_labels_for_row.append(label)
        mutated_y.append(new_labels_for_row)
    return np.array(mutated_y)

In [0]:
ltrdict = {'a':[1,0,0,0],
           'c':[0,1,0,0],
           'g':[0,0,1,0],
           't':[0,0,0,1],
           'n':[0,0,0,0],
           'A':[1,0,0,0],
           'C':[0,1,0,0],
           'G':[0,0,1,0],
           'T':[0,0,0,1],
           'N':[0,0,0,0]}

def onehot_encode(sequence):
    return np.array([ltrdict[x] for x in sequence])

x = np.array([onehot_encode(x) for x in sequences]) 

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [0]:
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        #divide by sqrt 2 for variance preservation
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])/(1.41421356237)
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))


In [0]:
kernel_size = 15
filters= 15
input_length = 1000

In [0]:
seed_num = 10000

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History, ModelCheckpoint

seed(seed_num)
set_random_seed(seed_num)

In [0]:
from keras.layers import Input
from keras.models import Model
from keras.models import load_model

In [0]:
filename = ('Seed %s.txt' % seed_num, str(seed_num))[0]
f = open(filename, 'w')

In [0]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

import keras.backend as K
# for some model with dropout ...

def predict_with_uncertainty(f, x, no_classes, n_iter=100):
    result = np.zeros((n_iter,) + (x.shape[0], no_classes) )

    for i in range(n_iter):
        result[i,:, :] = f((x, 1))[0]

    prediction = result.mean(axis=0)
    uncertainty = result.std(axis=0)
    return prediction     
  
def evaluate(model, x, y): 
  y_pred = model.predict(x)
  auroc = roc_auc_score(y, model.predict(x)) 
  auprc = average_precision_score(y, model.predict(x))
  print("auroc: " + str(auroc))
  print("auprc: "+ str(auprc))
  f.write("auroc: " + str(auroc) + "\n")
  f.write("auprc: " + str(auprc) + "\n")

    
def evaluate_dropout(model, x, y): 
  f2 = K.function([model.layers[0].input, K.learning_phase()],
               [model.layers[-1].output])
  y_pred = predict_with_uncertainty(f2, x, 2, n_iter = 100) 
  auroc = roc_auc_score(y, y_pred) 
  auprc = average_precision_score(y, y_pred)
  print("auroc: " + str(auroc))
  print("auprc: "+ str(auprc))
  f.write("auroc: " + str(auroc) + "\n")
  f.write("auprc: " + str(auprc) + "\n")

In [0]:
class RevCompSpatialDropout1D(Dropout): 
    def __init__(self, rate,**kwargs): 
        super(RevCompSpatialDropout1D, self).__init__(rate, **kwargs)
        self.seed = 3
        self.input_spec = InputSpec(ndim = 3)

    def _get_noise_shape(self, inputs): 
        input_shape = K.shape(inputs)
        noise_shape = (input_shape[0], 1, 1, int(self.num_input_chan/2)) 
        return noise_shape
        
    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        self.input_len = input_shape[1]
        super(RevCompSpatialDropout1D, self).build(input_shape)

    def call(self, inputs, training=None): 
        inputs_fwdandrevconcat = K.concatenate(
                tensors = [
                    inputs[:,:,None,:int(self.num_input_chan/2)],
                    inputs[:,:,None,int(self.num_input_chan/2):][:,:,:,::-1]],
                axis=2)

        if 0. < self.rate < 1.: 
            noise_shape = self._get_noise_shape(inputs)
            def dropped_inputs(): 
                dropped = K.dropout(inputs_fwdandrevconcat,
                                    self.rate, noise_shape, seed = self.seed)
                dropped = K.reshape(dropped, (-1, int(self.input_len), int(self.num_input_chan)))
                return K.concatenate(
                    tensors = [
                        dropped[:,:,:int(self.num_input_chan/2)],
                        dropped[:,:,int(self.num_input_chan/2):][:,:,::-1]],
                    axis=-1)

            return K.in_train_phase(dropped_inputs, inputs, training = training)

        return inputs

In [0]:
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import ops
import numbers
from tensorflow.python.framework import tensor_util
def _get_noise_shape(x, noise_shape):
  # If noise_shape is none return immediately.
  if noise_shape is None:
    return array_ops.shape(x)

  try:
    # Best effort to figure out the intended shape.
    # If not possible, let the op to handle it.
    # In eager mode exception will show up.
    noise_shape_ = tensor_shape.as_shape(noise_shape)
  except (TypeError, ValueError):
    return noise_shape

  if x.shape.dims is not None and len(x.shape.dims) == len(noise_shape_.dims):
    new_dims = []
    for i, dim in enumerate(x.shape.dims):
      if noise_shape_.dims[i].value is None and dim.value is not None:
        new_dims.append(dim.value)
      else:
        new_dims.append(noise_shape_.dims[i].value)
    return tensor_shape.TensorShape(new_dims)

  return noise_shape

class MCRCDropout(Layer):
    """Applies MC Dropout to the input.
       The applied noise vector is symmetric to reverse complement symmetry
       Class structure only slightly adapted 
    Dropout consists in randomly setting
    a fraction `rate` of input units to 0 at each update during training time,
    which helps prevent overfitting.
    Remains active ative at test time so sampling is required
    # Arguments
        rate: float between 0 and 1. Fraction of the input units to drop.
        noise_shape: 1D integer tensor representing the shape of the
            binary dropout mask that will be multiplied with the input.
            For instance, if your inputs have shape
            `(batch_size, timesteps, features)` and
            you want the dropout mask to be the same for all timesteps,
            you can use `noise_shape=(batch_size, 1, features)`.
        seed: A Python integer to use as random seed.
    # References
        - [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)
    """
    def __init__(self, rate, noise_shape=None, seed=None, **kwargs):
        super(MCRCDropout, self).__init__(**kwargs)
        self.rate = min(1., max(0., rate))
        self.noise_shape = noise_shape
        self.seed = seed
        self.supports_masking = True
        
    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(MCRCDropout, self).build(input_shape)

    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape

        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            import numpy as np
            noise_shape = self._get_noise_shape(inputs)
            x = inputs
            seed = self.seed
            keep_prob = 1. - self.rate
            if seed is None:
                seed = np.random.randint(10e6)
            # the dummy 1. works around a TF bug
            # (float32_ref vs. float32 incompatibility)
            x= x*1
            name = None
            with ops.name_scope(name, "dropout", [x]) as name:
                x = ops.convert_to_tensor(x, name="x")
                if not x.dtype.is_floating:
                    raise ValueError("x has to be a floating point tensor since it's going to"
                       " be scaled. Got a %s tensor instead." % x.dtype)
                if isinstance(keep_prob, numbers.Real) and not 0 < keep_prob <= 1:
                    raise ValueError("keep_prob must be a scalar tensor or a float in the "
                       "range (0, 1], got %g" % keep_prob)
                keep_prob = ops.convert_to_tensor(
                             keep_prob, dtype=x.dtype, name="keep_prob")
                keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

                # Do nothing if we know keep_prob == 1
                if tensor_util.constant_value(keep_prob) == 1:
                    return x

                noise_shape = _get_noise_shape(x, noise_shape)
                # uniform [keep_prob, 1.0 + keep_prob)
                random_tensor = keep_prob
                random_tensor += random_ops.random_uniform(
                noise_shape, seed=seed, dtype=x.dtype)
               
                # 0. if [keep_prob, 1.0) and 1. if [1.0, 1.0 + keep_prob)
                binary_tensor = math_ops.floor(random_tensor)
                dim = binary_tensor.shape[2]//2

                symmetric_binary = K.concatenate(
                    tensors = [
                      binary_tensor[:,:,int(self.num_input_chan/2):], 
                      binary_tensor[:,:,int(self.num_input_chan/2):][::,::-1,::-1]], 
                  axis=2)
                ret = math_ops.div(x, keep_prob) * symmetric_binary
                
                return ret


    def get_config(self):
        config = {'rate': self.rate,
                  'noise_shape': self.noise_shape,
                  'seed': self.seed}
        base_config = super(MCRCDropout, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

In [0]:
scale = 1.0
rc_model_standard_spatial_dropout = keras.models.Sequential()
rc_model_standard_spatial_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=x_train.shape[1:], padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_spatial_dropout.add(k1.core.Activation("relu"))
rc_model_standard_spatial_dropout.add(RevCompSpatialDropout1D(0.2))
rc_model_standard_spatial_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_spatial_dropout.add(k1.core.Activation("relu"))
rc_model_standard_spatial_dropout.add(RevCompSpatialDropout1D(0.2))
rc_model_standard_spatial_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_spatial_dropout.add(k1.core.Activation("relu"))
rc_model_standard_spatial_dropout.add(RevCompSumPool())
rc_model_standard_spatial_dropout.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard_spatial_dropout.add(Flatten())
rc_model_standard_spatial_dropout.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard_spatial_dropout.add(keras_genomics.layers.core.Dense(units = 2))
rc_model_standard_spatial_dropout.add(k1.core.Activation("sigmoid"))

In [368]:
rc_model_standard_spatial_dropout.compile(optimizer="adam",
                                          loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_rc_standard_spatial_dropout = rc_model_standard_spatial_dropout.fit(x_train, y_train, validation_split=0.2,  
                    callbacks= [early_stopping_callback], batch_size=100,  epochs=100)
rc_model_standard_spatial_dropout.set_weights(early_stopping_callback.best_weights)


rc_standard_spatial_dropout_filename = ('rc_standard_spatial_dropout_%s.h5' % seed_num, str(seed_num))[0]
rc_model_standard_spatial_dropout.save(rc_standard_spatial_dropout_filename)
custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D, 
                  'RevCompSumPool':RevCompSumPool, 
                  'RevCompSpatialDropout1D':RevCompSpatialDropout1D}

Train on 6000 samples, validate on 1500 samples
Epoch 1/100
6000/6000 [==============================] - 25s 4ms/step - loss: 0.7631 - acc: 0.5024 - val_loss: 0.6940 - val_acc: 0.5077
Epoch 2/100
6000/6000 [==============================] - 3s 515us/step - loss: 0.6947 - acc: 0.5060 - val_loss: 0.6941 - val_acc: 0.4957
Epoch 3/100
6000/6000 [==============================] - 3s 515us/step - loss: 0.6943 - acc: 0.5078 - val_loss: 0.6936 - val_acc: 0.5027
Epoch 4/100
6000/6000 [==============================] - 3s 511us/step - loss: 0.6932 - acc: 0.5140 - val_loss: 0.6930 - val_acc: 0.5190
Epoch 5/100
6000/6000 [==============================] - 3s 506us/step - loss: 0.6930 - acc: 0.5097 - val_loss: 0.6927 - val_acc: 0.5047
Epoch 6/100
6000/6000 [==============================] - 3s 506us/step - loss: 0.6931 - acc: 0.5150 - val_loss: 0.6923 - val_acc: 0.5137
Epoch 7/100
6000/6000 [==============================] - 3s 508us/step - loss: 0.6925 - acc: 0.5158 - val_loss: 0.6921 - val_acc: 0

In [0]:
rc_standard_model_final = load_model(rc_standard_spatial_dropout_filename, custom_objects)

In [0]:
scale = 1.0
rc_model_standard = keras.models.Sequential()
rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=x_train.shape[1:], padding="same"))
rc_model_standard.add(k1.core.Activation("relu"))
rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
rc_model_standard.add(k1.core.Activation("relu"))
rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
rc_model_standard.add(k1.core.Activation("relu"))
rc_model_standard.add(RevCompSumPool())
rc_model_standard.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard.add(Flatten())
rc_model_standard.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard.add(keras_genomics.layers.core.Dense(units = 2))
rc_model_standard.add(k1.core.Activation("sigmoid"))

In [371]:
rc_model_standard.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_rc_standard = rc_model_standard.fit(x_train, y_train, validation_split=0.2,  
                    callbacks= [early_stopping_callback], batch_size=100,  epochs=100)
rc_model_standard.set_weights(early_stopping_callback.best_weights)


rc_standard_filename = ('rc_standard_%s.h5' % seed_num, str(seed_num))[0]
rc_model_standard.save(rc_standard_filename)
custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D, 
                  'RevCompSumPool':RevCompSumPool}
rc_standard_model_final = load_model(rc_standard_filename, custom_objects)

Train on 6000 samples, validate on 1500 samples
Epoch 1/100
6000/6000 [==============================] - 25s 4ms/step - loss: 0.7522 - acc: 0.5097 - val_loss: 0.6947 - val_acc: 0.5083
Epoch 2/100
6000/6000 [==============================] - 3s 463us/step - loss: 0.6913 - acc: 0.5252 - val_loss: 0.6927 - val_acc: 0.5173
Epoch 3/100
6000/6000 [==============================] - 3s 437us/step - loss: 0.6863 - acc: 0.5543 - val_loss: 0.6927 - val_acc: 0.5237
Epoch 4/100
6000/6000 [==============================] - 3s 434us/step - loss: 0.6819 - acc: 0.5662 - val_loss: 0.6921 - val_acc: 0.5203
Epoch 5/100
6000/6000 [==============================] - 3s 427us/step - loss: 0.6749 - acc: 0.5854 - val_loss: 0.6897 - val_acc: 0.5320
Epoch 6/100
6000/6000 [==============================] - 3s 429us/step - loss: 0.6293 - acc: 0.6485 - val_loss: 0.5771 - val_acc: 0.6597
Epoch 7/100
6000/6000 [==============================] - 3s 427us/step - loss: 0.4891 - acc: 0.7265 - val_loss: 0.4591 - val_acc: 0

In [0]:
scale = 1.0
rc_model_standard_mcdropout = keras.models.Sequential()
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=x_train.shape[1:], padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(MCRCDropout(0.2))
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(MCRCDropout(0.2))
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(RevCompSumPool())
rc_model_standard_mcdropout.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard_mcdropout.add(Flatten())
rc_model_standard_mcdropout.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard_mcdropout.add(keras_genomics.layers.core.Dense(units = 2))
rc_model_standard_mcdropout.add(k1.core.Activation("sigmoid"))

In [373]:
rc_model_standard_mcdropout.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_rc_standard_mcdropout = rc_model_standard_mcdropout.fit(x_train, y_train, validation_split=0.2,  
                    callbacks= [early_stopping_callback], batch_size=100,  epochs=100)
rc_model_standard_mcdropout.set_weights(early_stopping_callback.best_weights)


rc_standard_filename_mcdropout = ('rc_standard_mcdropout_%s.h5' % seed_num, str(seed_num))[0]
rc_model_standard_mcdropout.save(rc_standard_filename_mcdropout)
custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D, 
                  'RevCompSumPool':RevCompSumPool,
                  'MCRCDropout':MCRCDropout}
rc_standard_model_final = load_model(rc_standard_filename, custom_objects)

Train on 6000 samples, validate on 1500 samples
Epoch 1/100
6000/6000 [==============================] - 26s 4ms/step - loss: 0.7860 - acc: 0.5065 - val_loss: 0.6943 - val_acc: 0.5100
Epoch 2/100
6000/6000 [==============================] - 3s 469us/step - loss: 0.6950 - acc: 0.5095 - val_loss: 0.6953 - val_acc: 0.5123
Epoch 3/100
6000/6000 [==============================] - 3s 464us/step - loss: 0.6948 - acc: 0.4998 - val_loss: 0.6962 - val_acc: 0.5060
Epoch 4/100
6000/6000 [==============================] - 3s 457us/step - loss: 0.6936 - acc: 0.5163 - val_loss: 0.6929 - val_acc: 0.5083
Epoch 5/100
6000/6000 [==============================] - 3s 470us/step - loss: 0.6937 - acc: 0.5167 - val_loss: 0.6936 - val_acc: 0.5187
Epoch 6/100
6000/6000 [==============================] - 3s 478us/step - loss: 0.6924 - acc: 0.5191 - val_loss: 0.6929 - val_acc: 0.5087
Epoch 7/100
6000/6000 [==============================] - 3s 471us/step - loss: 0.6926 - acc: 0.5130 - val_loss: 0.6918 - val_acc: 0

In [0]:
x_train_augment = np.asarray([val for val in x_train for __ in (0,1)])
for i in range(len(x_train_augment)):
    if i % 2 == 1:
        x_train_augment[i] = np.flip(x_train_augment[i])

x_test_augment = np.asarray([val for val in x_test for __ in (0,1)])
for i in range(len(x_test_augment)):
    if i % 2 == 1:
        x_test_augment[i] = np.flip(x_test_augment[i])
y_train_augment = np.asarray([val for val in y_train for __ in (0,1)])        
y_test_augment = np.asarray([val for val in y_test for __ in (0,1)])

In [375]:
augment_model = keras.models.Sequential()
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=(input_length,4), padding="same"))
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40))
augment_model.add(Flatten())
augment_model.add(k1.Dense(units = 100, activation = "relu"))
augment_model.add(k1.Dense(units = 2))
augment_model.add(k1.core.Activation("sigmoid"))

augment_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_augment = augment_model.fit(x_train_augment, y_train_augment, validation_split = 0.2, 
                           callbacks=[early_stopping_callback], batch_size = 200, epochs = 100)

augment_model.set_weights(early_stopping_callback.best_weights)
augment_filename = ('augment_%s.h5' % seed_num, str(seed_num))[0]
augment_model.save(augment_filename)
augment_final = load_model(augment_filename)

Train on 12000 samples, validate on 3000 samples
Epoch 1/100
12000/12000 [==============================] - 27s 2ms/step - loss: 0.6946 - acc: 0.5028 - val_loss: 0.6934 - val_acc: 0.5047
Epoch 2/100
12000/12000 [==============================] - 3s 285us/step - loss: 0.6926 - acc: 0.5135 - val_loss: 0.6932 - val_acc: 0.5072
Epoch 3/100
12000/12000 [==============================] - 3s 264us/step - loss: 0.6912 - acc: 0.5273 - val_loss: 0.6928 - val_acc: 0.5097
Epoch 4/100
12000/12000 [==============================] - 3s 249us/step - loss: 0.6827 - acc: 0.5612 - val_loss: 0.6601 - val_acc: 0.6168
Epoch 5/100
12000/12000 [==============================] - 3s 250us/step - loss: 0.5385 - acc: 0.7249 - val_loss: 0.4186 - val_acc: 0.7993
Epoch 6/100
12000/12000 [==============================] - 3s 251us/step - loss: 0.3595 - acc: 0.8368 - val_loss: 0.3086 - val_acc: 0.8622
Epoch 7/100
12000/12000 [==============================] - 3s 250us/step - loss: 0.2923 - acc: 0.8731 - val_loss: 0.27

In [376]:
augment_model_dropout = keras.models.Sequential()
augment_model_dropout.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=(input_length,4), padding="same"))
augment_model_dropout.add(k1.core.Activation("relu"))
augment_model_dropout.add(k1.Dropout(0.2))
augment_model_dropout.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
augment_model_dropout.add(k1.core.Activation("relu"))
augment_model_dropout.add(k1.Dropout(0.2))
augment_model_dropout.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
augment_model_dropout.add(k1.core.Activation("relu"))
augment_model_dropout.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40))
augment_model_dropout.add(Flatten())
augment_model_dropout.add(k1.Dense(units = 100, activation = "relu"))
augment_model_dropout.add(k1.Dense(units = 2))
augment_model_dropout.add(k1.core.Activation("sigmoid"))

augment_model_dropout.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_augment = augment_model_dropout.fit(x_train_augment, y_train_augment, validation_split = 0.2, 
                           callbacks=[early_stopping_callback], batch_size = 200, epochs = 100)

augment_model_dropout.set_weights(early_stopping_callback.best_weights)
augment_filename_dropout = ('augment_dropout_%s.h5' % seed_num, str(seed_num))[0]
augment_model_dropout.save(augment_filename_dropout)
augment_final_dropout = load_model(augment_filename_dropout)

Train on 12000 samples, validate on 3000 samples
Epoch 1/100
12000/12000 [==============================] - 27s 2ms/step - loss: 0.6957 - acc: 0.5036 - val_loss: 0.6929 - val_acc: 0.5097
Epoch 2/100
12000/12000 [==============================] - 3s 257us/step - loss: 0.6933 - acc: 0.5093 - val_loss: 0.6929 - val_acc: 0.5073
Epoch 3/100
12000/12000 [==============================] - 3s 228us/step - loss: 0.6932 - acc: 0.5046 - val_loss: 0.6925 - val_acc: 0.5115
Epoch 4/100
12000/12000 [==============================] - 3s 236us/step - loss: 0.6916 - acc: 0.5271 - val_loss: 0.6870 - val_acc: 0.5738
Epoch 5/100
12000/12000 [==============================] - 3s 243us/step - loss: 0.6211 - acc: 0.6193 - val_loss: 0.4969 - val_acc: 0.7025
Epoch 6/100
12000/12000 [==============================] - 3s 241us/step - loss: 0.4838 - acc: 0.7026 - val_loss: 0.4479 - val_acc: 0.7180
Epoch 7/100
12000/12000 [==============================] - 3s 242us/step - loss: 0.4468 - acc: 0.7210 - val_loss: 0.43

In [377]:
print("Reverse Complement/Standard Initialization")
f.write("Reverse Complement/Standard\n")
evaluate(rc_model_standard, x_test, y_test)

Reverse Complement/Standard Initialization
auroc: 0.9610226683709612
auprc: 0.9689409058931491


In [378]:
print("Reverse Complement/Standard Initialization/SpatialDropout")
f.write("Reverse Complement/Standard Initialization/SpatialDropout\n")
evaluate(rc_model_standard_spatial_dropout, x_test, y_test)
evaluate_dropout(rc_model_standard_spatial_dropout, x_test, y_test)

Reverse Complement/Standard Initialization/SpatialDropout
auroc: 0.968142417259965
auprc: 0.974471197706529
auroc: 0.9698301780108003
auprc: 0.9755099225835948


In [379]:
print("Reverse Complement/Standard Initialization/MCRCDropout")
f.write("Reverse Complement/Standard Initialization/MCRCDropout\n")
evaluate(rc_model_standard_mcdropout, x_test, y_test)
evaluate_dropout(rc_model_standard_mcdropout, x_test, y_test)

Reverse Complement/Standard Initialization/MCRCDropout
auroc: 0.7693913483841699
auprc: 0.7736342782705924
auroc: 0.7870974560196327
auprc: 0.7824637668493288


In [380]:
print("Augmented Model")
f.write("Augmented Model\n")
evaluate(augment_model, x_test_augment, y_test_augment)

Augmented Model
auroc: 0.9715124356256262
auprc: 0.9769896528564931


In [381]:
print("Augmented Model/Dropout")
f.write("Augmented Model/Dropout\n")
evaluate(augment_model, x_test_augment, y_test_augment)
evaluate_dropout(augment_model, x_test_augment, y_test_augment)

Augmented Model/Dropout
auroc: 0.9715124356256262
auprc: 0.9769896528564931
auroc: 0.9715125557252416
auprc: 0.9769897792342594


In [0]:
f.close()